## Preparation

In [ ]:
import ee
import geopandas as gpd
import pandas as pd
import os
import datetime
import numpy as np

In [ ]:
# Trigger the authentication flow.
# ee.Authenticate()

# Initialize the library.
ee.Initialize()

In [ ]:
# read inputs from spreadsheet
working_dr = os.getcwd()
data_dr = os.path.join(working_dr, 'data')
inputs_dr = os.path.join(working_dr, 'inputs.csv')
inputs_df = pd.read_csv(inputs_dr)

In [ ]:
# choose a season

# by index
index = 0
row = inputs_df.iloc[index]

# # by name
# name = '2020_2021_wheat'
# row = inputs_df.loc[inputs_df['season'] == name].iloc[0]

season = row['season']
project_shp_location = row['project_shp']
field_shp_location = row['field_shp']
start_date = row['start_date']
end_date = row['end_date']

In [ ]:
# convert shapefile to featurecollection
project_shp = gpd.read_file(project_shp_location) #read shapefile
field_shp = gpd.read_file(field_shp_location) #read shapefile
bbox = project_shp.total_bounds.tolist()
aoi = ee.Geometry.Rectangle(bbox)
geojson = field_shp.__geo_interface__
aoi_field = ee.FeatureCollection(geojson['features'])

## Get (ETa, NDVI, RGB) for each landsat image

In [ ]:
from geesebal import Image

In [ ]:
# landsat images
landsat_imgs = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')\
  .filterBounds(aoi)\
  .filter(ee.Filter.date(start_date, end_date)).distinct(['DATE_ACQUIRED'])
list_size = landsat_imgs.size().getInfo()
ls8_list = landsat_imgs.toList(list_size)
result_imgs = []
for i in range(list_size):
    img = ee.Image(ls8_list.get(i))
    landsat_img = Image(img.get('system:id').getInfo())
    result_imgs.append(landsat_img)
    print(f"{i+1} / {list_size} done")
print('all done')

## export tifs (RGB, NDIV, ETa)

In [ ]:
from funcs.export import export_tif_localy, export_to_drive

In [ ]:
shp_name = os.path.basename(project_shp_location).split('.')[0]
project_dr = os.path.join(working_dr, 'data', shp_name)
season_dr = os.path.join(working_dr, 'data', shp_name, f'season_{season}')
et_folder = os.path.join(season_dr, 'ET')
ndvi_folder = os.path.join(season_dr, 'ndvi')
rgb_folder = os.path.join(season_dr, 'rgb')

In [ ]:
# # export to drive
# shp_name = os.path.basename(project_shp_location).split('.')[0]
# for img in result_imgs:
#     # get img date
#     date = img.date_string
#     # export tif (to drive)
#     export_to_drive(img, date, os.path.join(shp_name, f'season_{season}'), aoi)

In [ ]:
# export localy
for img in result_imgs:
  date = img.date_string
  export_tif_localy(img.image, 'RGB', 'RGB', rgb_folder, date, aoi)
  export_tif_localy(img.image, 'ET_24h', 'ETa', et_folder, date, aoi)
  export_tif_localy(img.image, 'NDVI', 'ndvi', ndvi_folder, date, aoi)

## Daily ET dataframe

In [ ]:
from funcs.ETp import ETp_list

In [ ]:
# Get ETp for each date
# season days
s_d = datetime.datetime.strptime(start_date,'%Y-%m-%d')
e_d = datetime.datetime.strptime(end_date,'%Y-%m-%d')
days = [(s_d + datetime.timedelta(days=x)).strftime('%Y-%m-%d') for x in range((e_d-s_d).days + 2)]
daily_ETp_list = ETp_list(days, aoi)
days = days[:-1]
daily_ET_df = pd.DataFrame(
  {'date': pd.to_datetime(days) ,'ETp': daily_ETp_list, 'kc': np.nan, 'ETa': np.nan}
  )
os.makedirs(os.path.join(et_folder, "datasheets"), exist_ok= True)
daily_ET_df.to_csv(os.path.join(et_folder, "datasheets", "daily_ET.csv"))